In [1]:
import pickle
import numpy as np
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm
import os
import cv2
from flask import Flask, request, jsonify
from PIL import Image
from flask_cors import CORS
import requests
from io import BytesIO

In [2]:
app = Flask(__name__)
CORS(app)

In [3]:
feature_list=np.array(pickle.load(open('embeddings.pkl','rb')))
filenames=pickle.load(open('filenames.pkl','rb'))

In [4]:
model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable=False
model=tensorflow.keras.Sequential([model,GlobalMaxPooling2D()])

In [5]:
# img_path='sample/1.jpg'
# img=image.load_img(img_path,target_size=(224,224))
# img_array=image.img_to_array(img)
# expanded_img_array =np.expand_dims(img_array,axis=0)
# preprocess_img=preprocess_input(expanded_img_array)
# result=model.predict(preprocess_img).flatten()
# normalised_result=result/norm(result)

In [6]:
neighbors=NearestNeighbors(n_neighbors=6,algorithm='brute',metric='cosine')
neighbors.fit(feature_list)

# distances,indices=neighbors.kneighbors([normalised_result])
# for file in indices[0][1:6]:
# #     img_temp=cv2.imread(filenames[file])
# #     cv2.imshow('output',cv2.resize(img_temp,(512,512)))
# #     cv2.waitKey(0)
#     print(filenames[file])

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [7]:
@app.route('/get_recommendations', methods=['POST'])
def get_recommendations():
    try:
        # Get the image URL from the request
        image_url = request.form['image']
        # print(image_url)
        response = requests.get(image_url)
        img_temp = Image.open(BytesIO(response.content)).resize((224, 224))
        # print(img_temp,"image is resinging")    
        # img_array = np.array(img_temp)_
        img_array=image.img_to_array(img_temp)
        # print(img_array,"image is converted into array")
        expanded_img_array =np.expand_dims(img_array,axis=0)
        preprocess_img=preprocess_input(expanded_img_array)
        result=model.predict(preprocess_img).flatten()
        normalised_result=result/norm(result)
        normalised_result_reshaped = normalised_result.reshape(1, -1)

        print(normalised_result)
        distances, indices = neighbors.kneighbors(normalised_result_reshaped)
        recommended_files = [filenames[file] for file in indices[0][1:6]]
        # return jsonify({'success': True})
        print(recommended_files)

        return jsonify({'recommendations': recommended_files})
        # Rest of your code here using image_url
        # '''1st phase'''

        # img_temp = cv2.imdecode(np.frombuffer(response.content, np.uint8), -1)
        # cv2.imshow('output',cv2.resize(img_temp,(512,512)))
        # cv2.waitKey(0)
        # print(response.raw)

        # img = Image.open(img_temp).resize((224, 224))

        # img_array = np.expand_dims(image.img_to_array(img), axis=0)
        # preprocess_img = preprocess_input(img_array)
        # result = model.predict(preprocess_img).flatten()
        # normalised_result = result / norm(result)
        # print(normalised_result)
        # distances, indices = neighbors.kneighbors([normalised_result])
        # recommended_files = [filenames[file] for file in indices[0][1:6]]
        # # print(recommended_files)



        
        # return jsonify({'success': True})

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    try:
        print("Starting Flask app")
        app.run(debug=True, use_reloader=False)
    except Exception as e:
        print(f"An exception occurred: {e}")
        # Print the full traceback to get more details about the exception
        import traceback
        traceback.print_exc()

Starting Flask app
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 [==============================] - 4s 4s/step
[0.02308633 0.0303117  0.00409676 ... 0.0273345  0.01398251 0.01589241]
[0.02308633 0.0303117  0.00409676 ... 0.0273345  0.01398251 0.01589241]


127.0.0.1 - - [10/Mar/2024 16:06:40] "POST /get_recommendations HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2024 16:06:40] "POST /get_recommendations HTTP/1.1" 200 -


['images\\1533.jpg', 'images\\1625.jpg', 'images\\1537.jpg', 'images\\1995.jpg', 'images\\1563.jpg']['images\\1533.jpg', 'images\\1625.jpg', 'images\\1537.jpg', 'images\\1995.jpg', 'images\\1563.jpg']

1/1 [==============================] - 2s 2s/step
[0.02308633 0.0303117  0.00409676 ... 0.0273345  0.01398251 0.01589241][0.02308633 0.0303117  0.00409676 ... 0.0273345  0.01398251 0.01589241]



127.0.0.1 - - [10/Mar/2024 16:10:00] "POST /get_recommendations HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2024 16:10:00] "POST /get_recommendations HTTP/1.1" 200 -


['images\\1533.jpg', 'images\\1625.jpg', 'images\\1537.jpg', 'images\\1995.jpg', 'images\\1563.jpg']['images\\1533.jpg', 'images\\1625.jpg', 'images\\1537.jpg', 'images\\1995.jpg', 'images\\1563.jpg']



In [ ]:
# @app.route('/get_recommendations', methods=['POST'])
# def get_recommendations():
#     try:
#         # Get the image path from the request
#         # img_path = request.json['img_path']
        
#         # img_file = request.files['image']
#         print("hello")
#         img_url = request.form['image_Url']
#         print(image_url,"url image")
#         response = requests.get(img_url, stream=True)
#         response.raise_for_status()
#         # print("response",response)
#         print("Response status code:", response.status_code)
#         print("Response content:", response.content)

#         if response.status_code == 200:
#             # Open the image using PIL
#             img = Image.open(response.raw).resize((224, 224))

#             img_array = np.expand_dims(image.img_to_array(img), axis=0)
#             preprocess_img = preprocess_input(img_array)
#             result = model.predict(preprocess_img).flatten()
#             normalised_result = result / norm(result)

#             distances, indices = neighbors.kneighbors([normalised_result])
#             recommended_files = [filenames[file] for file in indices[0][1:6]]

#             return jsonify({'recommendations': recommended_files})
#         else:
#             return jsonify({'error': f'Failed to fetch image. HTTP status code: {response.status_code}'})


        # print(img_file)
        # img = Image.open(img_file)

        # # Resize the image to (224, 224)
        # resized_img = img.resize((224, 224))
        
        # print(resized_img)

        # img = Image.open(response.raw).resize((224, 224))
        
        # # Process the image and get the recommendation
        # img = image.load_img(img_path, target_size=(224, 224))
        # img_array = image.img_to_array(resized_img)
        # expanded_img_array = np.expand_dims(img_array, axis=0)
        # preprocess_img = preprocess_input(expanded_img_array)
        # result = model.predict(preprocess_img).flatten()
        # normalised_result = result / norm(result)
        
        # print(normalised_result)

        # distances, indices = neighbors.kneighbors([normalised_result])
        # recommended_files = [filenames[file] for file in indices[0][1:6]]
        # print(recommended_files,"hello")
        # return jsonify({'recommendations': recommended_files})

#     except Exception as e:
#         return jsonify({'error': str(e)})

# if __name__ == '__main__':
#     try:
#         print("Starting Flask app")
#         app.run(debug=True, use_reloader=False)
#     except Exception as e:
#         print(f"An exception occurred: {e}")
#         # Print the full traceback to get more details about the exception
#         import traceback
#         traceback.print_exc()

Starting Flask app
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


hellohello



127.0.0.1 - - [09/Mar/2024 12:59:07] "POST /get_recommendations HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2024 12:59:07] "POST /get_recommendations HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2024 13:01:35] "POST /get_recommendations HTTP/1.1" 200 -


hello


...

hello


127.0.0.1 - - [09/Mar/2024 13:07:58] "POST /get_recommendations HTTP/1.1" 200 -


hello


127.0.0.1 - - [09/Mar/2024 13:08:59] "POST /get_recommendations HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2024 13:08:59] "POST /get_recommendations HTTP/1.1" 200 -


hellohello



In [ ]:
%tb

SystemExit: 1